In [1]:
import pandas as pd
from datetime import datetime,timedelta
import math
import keras
from keras.wrappers.scikit_learn import KerasRegressor
import numpy as np
import src.functions as func
from src import feagen
from sklearn.svm import SVR

feat = feagen.Features(
        '../dataSets/training/',
        'weather (table 7)_training.csv',
        'volume(table 6)_training.csv',
        'trajectories(table 5)_training.csv'
    )


Using TensorFlow backend.


In [2]:


# Data Setting
train_dates = ('2016-09-19', '2016-10-10')
valid_dates = ('2016-10-11', '2016-10-17')
test_dates = ('2016-10-18', '2016-10-24')


vol_in_dim = 79
tra_in_dim = 44


# Model Setting
hidden_layer = 1
hidden_dim = [20]
act_func = ['relu']

# Training Parameters
epoch = 1000
batch = 128



In [3]:

tr_x, tr_y = feat.get_vol_Xy(dates=train_dates, ampm='am')
print sum(np.isnan(tr_x))
tr_x = np.nan_to_num(tr_x)

#clf = SVR(C=1.0, epsilon=0.2)
#clf.fit(tr_x, tr_y)
am_vol_NN = func.gen_model(vol_in_dim,hidden_layer,hidden_dim,act_func)
am_vol_NN.compile(loss='mse', optimizer='adam')
am_vol_NN.fit(tr_x,tr_y,nb_epoch=epoch, batch_size=batch,verbose=0)


tr_x, tr_y = feat.get_vol_Xy(dates=train_dates, ampm='pm')
#print sum(tr_x<0)
pm_vol_NN = func.gen_model(vol_in_dim,hidden_layer,hidden_dim,act_func)
pm_vol_NN.compile(loss='mse', optimizer='adam')
pm_vol_NN.fit(tr_x,tr_y,nb_epoch=epoch, batch_size=batch,verbose=0)

tr_x, tr_y = feat.get_tra_Xy(dates=train_dates, ampm='am')
#print sum(np.isnan(tr_x))
am_tra_NN = func.gen_model(tra_in_dim,hidden_layer,hidden_dim,act_func)
am_tra_NN.compile(loss='mse', optimizer='adam')
am_tra_NN.fit(tr_x,tr_y,nb_epoch=epoch, batch_size=batch,verbose=0)

tr_x, tr_y = feat.get_tra_Xy(dates=train_dates, ampm='pm')
print tr_x
pm_tra_NN = func.gen_model(tra_in_dim,hidden_layer,hidden_dim,act_func)
pm_tra_NN.compile(loss='mse', optimizer='adam')
pm_tra_NN.fit(tr_x,tr_y,nb_epoch=epoch, batch_size=batch,verbose=0)



[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 12  0  0 12  0  0 12  0  0
  6  0  0  6  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0]
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  1.  0.  0.]
 [ 1.  0.  0. ...,  0.  1.  0.]
 [ 1.  0.  0. ...,  0.  0.  1.]]


In [64]:
val_x, val_y = feat.get_vol_Xy(dates=valid_dates, ampm='am')
am_vol_val,am_vol_c = func.my_mape(am_vol_NN.predict(val_x).flatten(),val_y)



val_x, val_y = feat.get_vol_Xy(dates=valid_dates, ampm='pm')
pm_vol_val,pm_vol_c = func.my_mape(pm_vol_NN.predict(val_x).flatten(),val_y)
#pm_vol_val,pm_vol_c = func.my_mape(clf.predict(val_x).flatten(),val_y)

val_x, val_y = feat.get_tra_Xy(dates=valid_dates, ampm='am')
am_tra_val,am_tra_c = func.my_mape(am_tra_NN.predict(val_x).flatten(),val_y)

val_x, val_y = feat.get_tra_Xy(dates=valid_dates, ampm='pm')
pm_tra_val,pm_tra_c = func.my_mape(pm_tra_NN.predict(val_x).flatten(),val_y)


print 'Volume AM valid score :',am_vol_val
print 'Volume PM valid score :',pm_vol_val
print 'Volume over all valid score :',((am_vol_val*am_vol_c)+(pm_vol_val*pm_vol_c))/(am_vol_c+pm_vol_c)
print 'Traj AM valid score :',am_tra_val
print 'Traj PM valid score :',pm_tra_val
print 'Traj overall valid score',((am_tra_val*am_tra_c)+(pm_tra_val*pm_tra_c))/(am_tra_c+pm_tra_c)

Volume AM valid score : 0.186445515901
Volume PM valid score : 37.4880045302
Volume over all valid score : 18.837225023
Traj AM valid score : 0.248038181254
Traj PM valid score : 90.7152941022
Traj overall valid score 45.0795894488


# Testing Data

In [ ]:

time_delta = timedelta(minutes=20)


vol_tolls = ['1', '1', '2', '3', '3']
vol_dires = ['0', '1', '0', '0', '1']
toll_dire = zip(vol_tolls, vol_dires)

tra_intes = ['A', 'A', 'B', 'B', 'C', 'C']
tra_tolls = [2, 3, 1, 3, 1, 3]
inte_toll = zip(tra_intes, tra_tolls)


with open('volume_pred.csv','w') as pred_vol:
    ans = []
    #predict am vol
    test_dates = datetime.strptime('2016-10-18 08:00:00', "%Y-%m-%d %H:%M:%S")
    ttx = get_vol_X(dates=valid_dates, ampm='am')
    tty = am_vol_NN.predict(ttx)
    i = 0
    for tol,dire in toll_dire:
        for day in range(7):
            for w in range(6):
                ins = []
                ins.append(tol)
                ins.append('['+str(test_dates+w*time_delta)+','+str(test_dates+(w+1)*time_delta)+')')
                ins.append(dire)
                ins.append(tty[i])
                i += 1
                ans.append(ins)
        test_dates += timedelta(days=1)
    #predict pm vol
    test_dates = datetime.strptime('2016-10-18 17:00:00', "%Y-%m-%d %H:%M:%S")
    ttx = get_vol_X(dates=valid_dates, ampm='pm')
    tty = pm_vol_NN.predict(ttx)
    i = 0
    for tol,dire in toll_dire:
        for day in range(7):
            for w in range(6):
                ins = []
                ins.append(tol)
                ins.append('['+str(test_dates+w*time_delta)+','+str(test_dates+(w+1)*time_delta)+')')
                ins.append(dire)
                ins.append(tty[i])
                i += 1
                ans.append(ins)
        test_dates += timedelta(days=1)
        
        
            
    
    
with open('travel_time_pred.csv','w') as pred_tra:
    ans = []
    #predict am tra
    test_dates = datetime.strptime('2016-10-18 08:00:00', "%Y-%m-%d %H:%M:%S")
    ttx = get_tra_X(dates=valid_dates, ampm='am')
    tty = am_tra_NN.predict(ttx)
    i = 0 
    for inte,tol in inte_toll:
        for day in range(7):
            for w in range(6):
                ins = []
                ins.append(inte)
                ins.append(tol)
                ins.append('['+str(test_dates+w*time_delta)+','+str(test_dates+(w+1)*time_delta)+')')
                ins.append(tty[i])
                i += 1
                ans.append(ins)
        test_dates += timedelta(days=1)
    #predict pm tra
    test_dates = datetime.strptime('2016-10-18 17:00:00', "%Y-%m-%d %H:%M:%S")
    ttx = get_tra_X(dates=valid_dates, ampm='pm')
    tty = am_tra_NN.predict(ttx)
    i = 0 
    for inte,tol in inte_toll:
        for day in range(7):
            for w in range(6):
                ins = []
                ins.append(inte)
                ins.append(tol)
                ins.append('['+str(test_dates+w*time_delta)+','+str(test_dates+(w+1)*time_delta)+')')
                ins.append(tty[i])
                i += 1
                ans.append(ins)
        test_dates += timedelta(days=1)

In [7]:
d = {'col1': 'vol', 'col2': 'tar'}
df = pd.DataFrame(data=d, index=[0,1])
print df

  col1 col2
0  vol  tar
1  vol  tar
